<a href="https://colab.research.google.com/github/nebuchad-nezzar/Quantiative-Finance/blob/main/Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 36.2 MB/s eta 0:00:00


In [51]:
import numpy as np
import yfinance as yf
from arch import arch_model
import matplotlib.pyplot as plt
from scipy.stats import norm
import plotly.graph_objects as go

def forecast_volatility(data):
    # Use 'Close' if 'Adj Close' is not available
    if 'Adj Close' in data.columns:
        prices = data['Adj Close']
    elif 'Close' in data.columns:
        prices = data['Close']
    else:
        raise KeyError("Neither 'Adj Close' nor 'Close' column found in the data.")

    returns = 100 * prices.pct_change().dropna()
    model = arch_model(returns)
    res = model.fit(last_obs="2025-01-17", update_freq=5)

    # Get the variance forecast
    forecast = res.forecast(horizon=30, reindex=False)
    variance_forecast = forecast.variance.iloc[-1][0]

    # Compute the annualized volatility forecast
    volatility_forecast = np.sqrt(variance_forecast)
    annualized_volatility_forecast = volatility_forecast * np.sqrt(252) / 100

    # Create a Plotly figure for the volatility forecast
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=res.conditional_volatility.index, y=res.conditional_volatility, mode='lines', name='Conditional Volatility'))
    fig.update_layout(
        title="Volatility Forecast",
        xaxis_title="Date",
        yaxis_title="Volatility",
    )
    fig.show()

    print(res.summary())
    return annualized_volatility_forecast

class BsOption:
    def __init__(self, S, K, T, r, sigma, q):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.q = q

    @staticmethod
    def N(x):
        return norm.cdf(x)

    @property
    def params(self):
        return {'S': self.S,
                'K': self.K,
                'T': self.T,
                'r':self.r,
                'q':self.q,
                'sigma':self.sigma}

    def d1(self):
        return (np.log(self.S/self.K) + (self.r -self.q + self.sigma**2/2)*self.T) \
                                / (self.sigma*np.sqrt(self.T))

    def d2(self):
        return self.d1() - self.sigma*np.sqrt(self.T)

    def _call_value(self):
        return self.S*np.exp(-self.q*self.T)*self.N(self.d1()) - \
                    self.K*np.exp(-self.r*self.T) * self.N(self.d2())

    def _put_value(self):
        return self.K*np.exp(-self.r*self.T) * self.N(-self.d2()) -\
                self.S*np.exp(-self.q*self.T)*self.N(-self.d1())

    def price(self, type_ = 'C'):
        if type_ == 'C':
            return self._call_value()
        if type_ == 'P':
            return self._put_value()
        if type_ == 'B':
            return  {'call': self._call_value(), 'put': self._put_value()}
        else:
            raise ValueError('Unrecognized type')

# Download data and check for 'Adj Close' or 'Close'
data = yf.download("ARES", start="2020-01-01", end="2025-01-17")
print(data.head())  # Inspect the DataFrame
print(data.columns)  # Check column names

# Ensure 'Adj Close' or 'Close' exists before proceeding
if 'Adj Close' in data.columns or 'Close' in data.columns:
    sigma = forecast_volatility(data)
    print("Annualized Volatility Forecast:", sigma)
    print(BsOption( 194.81, 150.00,  (7/365), 0.048, sigma, 0).price('B'))
else:
    print("Error: Neither 'Adj Close' nor 'Close' column found. Check the ticker symbol and data range.")

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open  Volume
Ticker           ARES       ARES       ARES       ARES    ARES
Date                                                          
2020-01-02  30.190699  30.483769  29.825420  30.437050  506500
2020-01-03  29.935860  30.114251  29.782952  29.833919  401100
2020-01-06  30.233170  30.420057  29.816921  29.816921  387000
2020-01-07  30.105745  30.335107  29.999560  30.020798  349500
2020-01-08  30.674902  30.742859  30.105745  30.284140  683400
MultiIndex([( 'Close', 'ARES'),
            (  'High', 'ARES'),
            (   'Low', 'ARES'),
            (  'Open', 'ARES'),
            ('Volume', 'ARES')],
           names=['Price', 'Ticker'])
Iteration:      5,   Func. Count:     37,   Neg. LLF: 2696.418810794846
Iteration:     10,   Func. Count:     61,   Neg. LLF: 2696.4143115845172
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2696.4143115844713
            Iterations: 10
            Func


<ipython-input-51-f450054d6a16>:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



                     Constant Mean - GARCH Model Results                      
Dep. Variable:                   ARES   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2696.41
Distribution:                  Normal   AIC:                           5400.83
Method:            Maximum Likelihood   BIC:                           5421.41
                                        No. Observations:                 1267
Date:                Tue, Jan 21 2025   Df Residuals:                     1266
Time:                        20:18:23   Df Model:                            1
                               Mean Model                               
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.2128  5.136e-02      4.143  3.432e-05 [  0.112,  0.31